In [ ]:
import os
import shutil
import sys
import json
import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from io import BytesIO,StringIO
import jaydebeapi as jp
import warnings
warnings.filterwarnings('ignore')

import os 
os.chdir("/project_data/data_asset")
from hdfs.config import Config
from useful.db_conn import * 
from useful.libs import *  

In [ ]:
# hdfs client
config = Config()
config.add_section(config.global_section)
section = 'cdp.alias'
config.add_section(section)
config.set(section, 'url', 'http://~~~~~~~~~~~~/;http://~~~~~~~~~~~~')
config.set(section, 'user', '~~~~~~~~~~~~')
client = config.get_client('cdp')

In [ ]:
#pyspark
db = DB()
sp = SparkDB('hive_table')
sp.spark.conf.set('spark.sql.parquet.writeLegacyFormat', 'TRUE')
db = DB()
hconn = db.hconn
hcurr = hconn.cursor()

In [ ]:
sparkDataFrame = sp.spark.read.parquet(f'hdfs://datalake/user/~~~~~~~~~~~~/UploadData/ACRM/TSMFCRM70_ALL/20230315_crm70_all.parquet')
sparkDataFrame.count()
sparkDataFrame.rdd.getNumPartitions()
#print(sparkDataFrame.rdd.getNumPartitions())
sparkDataFrame.columns

In [ ]:
new_column_name = [
    'acmplcmpgnid',
    'sendpshistidnfr',
    'sendymd',
    'msgcretncd',
    'custidnfr',
    'custmgtno',
    'custnm',
    'sendbrncd',
    'sendempid',
    'notimsgmdiadstcd',
    'notimsgmdiadtalsdstcd',
    'rcverinfoctnt',
    'notisendstusdstcd',
    'sendmsgtitlctnt',
    'sendmsgctnt',
    'sendsvcdsticctnt',
    'sendsvcno',
    'bzwkrceptidnfiid',
    'syslastuno',
    'flbcmdiactnt',
    'flbcyn',
    'newstarbnkgpushlagclsfidstcd',
    'newstarbnkgpushmedmclsfidstcd',
    'nttksendstusdstcd',
    'nttksendrsultdstcd',
    'nttktmpltid',
    'frndtksendstusdstcd',
    'frndtksendrsultdstcd'
]
for i ,col_name in enumerate(sparkDataFrame.columns) :
    sparkDataFrame = sparkDataFrame.withColumnRenamed(col_name,new_column_name[i])

sparkDataFrame.printSchema()

In [ ]:

sparkDataFrame = sparkDataFrame.withColumn(
    'p_yyyymmdd'
    , F.col('sendymd')
)
sparkDataFrame = sparkDataFrame.drop("custnm")

In [ ]:
(
    sparkDataFrame
    .write
    .option('header', True)
    .partitionBy('p_yyyymmdd')
    .format('parquet')
    .mode('overwrite')
    .save(f'hdfs://datalake/user/~~~~/UploadTable/tsmfcrm70_all')
)

client.list(f'/user/~~~~~~~~~~~~/UploadTable/tsmfcrm70_all/')

In [ ]:
data_urls = client.list(base_url)[1:]
data_urls

base_url = '/user/~~~~~~~~~~~~/UploadTable/tsmfcrm70_all'
data_urls = client.list(base_url)[1:]

for p_yyyymmdd in data_urls:
    time.sleep(0.1)
    k1, v1 = p_yyyymmdd.split('=')[0], p_yyyymmdd.split('=')[1]

    hcurr.execute(
        f"""
            LOAD DATA INPATH '/user/~~~~~~~~~~~~/UploadTable/tsmfcrm70_all/{p_yyyymmdd}' 
            OVERWRITE INTO TABLE project_dma.tsmfcrm70 
            PARTITION({k1}='{v1}')
        """
    )